# 🧠 Agent Performance Optimization Engine
This notebook builds a machine learning model to predict agent performance.

In [ ]:

import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib


### 📥 Load Dataset

In [ ]:

df = pd.read_csv("../data/Customer_support_data.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')


### 🎯 Create Target Variable

In [ ]:

df['target'] = ((df['csat_score'] >= 4)).astype(int)


### ✨ Feature Selection & Preprocessing

In [ ]:

features = ['category', 'sub-category', 'agent_name', 'tenure_bucket', 'agent_shift', 'csat_score']
df_model = df[features + ['target']].dropna()

categorical_cols = ['category', 'sub-category', 'agent_name', 'tenure_bucket', 'agent_shift']
encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])
    encoders[col] = le

X = df_model.drop(['target'], axis=1)
y = df_model['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### 🤖 Train the Model

In [ ]:

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
print(classification_report(y_test, y_pred))


### 💾 Save the Model and Encoders

In [ ]:

os.makedirs("../app", exist_ok=True)
joblib.dump(model, "../app/model.pkl")
joblib.dump(encoders, "../app/encoder.pkl")
joblib.dump(scaler, "../app/scaler.pkl")
